## Data Cleaning and Feature Engineering  
Author: Anne Chen  
2016

### Import Modules

In [1]:
import pandas as pd
import numpy as np
import json
import re
import nltk
from nltk import *
from textblob import TextBlob
# NOTE:
# if this is the first time a user is using nltk
# he/she must run the following code: nltk.download('all')

### Functions
- Define Utility Functions

In [2]:
def read_json(filename):
    '''read json file'''
    return json.loads(open(filename).read()) 

In [3]:
def drop_first_column(us_museum, w_museum):
    '''drop the first column'''
    if us_museum.columns.values[0] != 'Address':
        us_museum = us_museum.drop(us_museum.columns[0], axis = 1)
        w_museum = w_museum.drop(w_museum.columns[0], axis = 1)
    return us_museum, w_museum

In [4]:
def merge_usa_world(us_museum, w_museum):    
    '''merge us and world dataframe'''
    merged_df = pd.merge(us_museum, w_museum, how='outer')
    merged_df = merged_df.drop_duplicates('MuseumName')
    return merged_df

In [47]:
def fix_fee_value(df):
    '''fix the fee value and replace yes/no with none in column "LengthOfVisit" '''    
    idx_no = np.where(df['LengthOfVisit'] == 'No ')[0].tolist()
    idx_yes = np.where(df['LengthOfVisit'] == 'Yes ')[0].tolist()
    df1 = df.copy(deep=True)
    for idx in idx_no:
        df1.loc[idx,'LengthOfVisit'] = None
        df1.loc[idx,'Fee'] = 'No '
    for idx in idx_yes:
        df1.loc[idx,'LengthOfVisit'] = None
        df1.loc[idx,'Fee'] = 'Yes '
    df = df1.copy(deep = True)
    return df

In [14]:
def string_to_num(df, column_name_lst):
    '''convert string in to number, e.g. 1,354 --> 1345'''
    for col_name in column_name_lst:
        df[col_name] = df[col_name].apply(lambda x: int(x.replace(',', '')))
    return df

In [15]:
def append_two_dict(dict1, dict2):
    '''append two dictionaries based on keys'''
    new_dict = dict1
    for key, val in dict2.items():
        if key not in dict1.keys():
            new_dict[key] = val
    return new_dict

In [16]:
def assign_0_or_1(df, target, dic):
    '''assign value as 1 if the values of the dictionary in the musuem match the target'''
    df = df.copy(deep=True)
    # initialize column with 0
    for sub_item in target:
        df[sub_item] = 0

    for museum_name, value in dic.items():
        # get index
        idx = df[df['MuseumName'] == museum_name].index.tolist()[0]

        for sub_item in target:
            if sub_item in value:
                df.loc[idx, sub_item] = 1  
    return df

In [20]:
def unicode_to_ascii(lst):
    '''convert unicode to ascii'''
    # avoid raising errors later on while writing data into csv files
    return [item.encode('ascii', 'ignore') for item in lst]

In [21]:
def encode_whole_dictionary(dic):
    '''convert whole dictionaty from unicode to ascii'''
    keys = dic.keys()
    values = dic.values()
    encode_key = unicode_to_ascii(keys)
    encode_val = [unicode_to_ascii(val) for val in values]
    
    # create new dictionary with encoded kay and values
    new_dic ={}
    for i in range(len(encode_key)):
        new_dic[encode_key[i]] = encode_val[i]

    return new_dic

In [27]:
def clean_dic(dic):
    '''clean the dictionary by removing the key-value pair with NaN'''
    return {k: dic[k] for k in dic if not isinstance(k, float)}

In [29]:
def dic_str_to_num(dic):
    '''convert list of strings in a dictionary into numbers'''
    new_dic = {}
    for key, val_lst in dic.items():
        new_dic[key] = [int(x.replace(',', '')) for x in val_lst]
    return new_dic

- Define Feature Engineering Related Functions

In [5]:
def add_country_state(df, state_name_lst):
    '''add new feature Country: USA or Other'''
    df['Country'] = 'Other'
    df['State'] = 'Not_in_USA'
    for idx, address in enumerate(df['Address']):
        try:
            state = address.split(',')[-1].split()[0]
            if state in state_name_lst:
                df.loc[idx,'Country'] = 'USA'
                df.loc[idx,'State'] = state
        except:
            pass

In [17]:
def add_category_feature(df, category):
    '''add several museum type as new features'''
    category_lst = reduce(lambda x,y: x + y, category.values(),[])
    target_category = [i for i in set(category_lst) if 'Museum' in i \
                       or 'Galleries' in i or 'Historic Sites' in i or 'Landmarks' in i]
    # print target_category
    df = assign_0_or_1(df, target_category, category)    
    return df

In [18]:
def get_tag_cloud_lst(tag_cloud):
    '''get tag cloud list: [(tag name, its frequency)]'''
    tag_cloud_lst = reduce(lambda x,y: x + y, tag_cloud.values(),[])
    freq_dic = {}
    for i in tag_cloud_lst:
        if i not in freq_dic.keys(): 
            freq_dic[i] = 1
        else:
            freq_dic[i] +=1
    tag_freq_lst = freq_dic.items()
    tag_freq_lst.sort(key = lambda x: x[1], reverse=True)
    return tag_freq_lst

In [19]:
def add_tags_feature(df, tag_cloud):
    '''add top 30 tags as features'''
    tag_freq_lst = get_tag_cloud_lst(tag_cloud)
    target_tags = map(lambda x: x[0], tag_freq_lst[0:30])
    df = assign_0_or_1(df, target_tags, tag_cloud)    
    return df    

In [22]:
def add_traveler_type_count(df):
    '''add review count for each traveler type as new feature'''
    df = df.copy(deep=True)
    type_of_traveler = ['Families_Count', 'Couples_Count', 'Solo_Count', 'Business_Count', 'Friends_Count']
    for idx, item in enumerate(type_of_traveler):    
        for museum_name, value in traveler_type.items():
            # get index
            row_idx = df[df['MuseumName'] == museum_name].index.tolist()[0]
            # assign review count for this traveler type 
            df.loc[row_idx, item] = traveler_type[museum_name][idx]
    return df

In [23]:
def create_description_dic(df):
    '''create dictionary --> {museum name: museum description}'''
    description_dic = {}
    for i in df.index:
        description_dic[df['MuseumName'][i]] = df['Description'][i]
    return description_dic

In [24]:
def get_nested_sentiment(dic):
    '''get polarity and subjectivity score for each text in nested list'''
    polarity_dic = {}
    subjectivity_dic = {}
    for museum_name, lst in dic.items():
        polarity_lst = []
        subjectivity_lst = []
        for sentence in lst:
            blob = TextBlob(sentence)
            polarity_lst.append(blob.sentiment.polarity)
            subjectivity_lst.append(blob.sentiment.subjectivity)
        polarity_dic[museum_name] = polarity_lst
        subjectivity_dic[museum_name] = subjectivity_lst
    return polarity_dic, subjectivity_dic

In [25]:
def get_text_sentiment(dic):
    '''get polarity and subjectivity score for text passing in'''
    polarity_dic = {}
    subjectivity_dic = {}
    for museum_name, text in dic.items():
        try:
            blob = TextBlob(text)
            polarity_dic[museum_name] = blob.sentiment.polarity
            subjectivity_dic[museum_name] = blob.sentiment.subjectivity
        except:
            polarity_dic[museum_name] = float('NaN')
            subjectivity_dic[museum_name] = float('NaN')
    return polarity_dic, subjectivity_dic

In [26]:
def add_multiple_score_feature(df, header, dic):
    '''add max, min, mean, and var of sentiment scores as new features'''
    df = df.copy(deep=True)   
    colnames = [header+'var', header+'mean', header+'max', header+'min']
    for museum_name, score_lst in dic.items():
        # get index
        row_idx = df[df['MuseumName'] == museum_name].index.tolist()[0]
        # create new features for scores: max, min, mean, and var of score_lst
        df.loc[row_idx, colnames[0]] = np.var(score_lst)
        df.loc[row_idx, colnames[1]] = np.mean(score_lst)
        df.loc[row_idx, colnames[2]] = max(score_lst)
        df.loc[row_idx, colnames[3]] = min(score_lst)
    return df

In [28]:
def add_one_feature(df, colname, dic):
    '''add the value in dic as new feature to df with colname as new column name'''
    df = df.copy(deep=True) 
    for museum_name, value in dic.items():
        # get index
        row_idx = df[df['MuseumName'] == museum_name].index.tolist()[0]
        # assign score
        df.loc[row_idx, colname] = value
    return df

In [30]:
def get_precise_rating(df, traveler_rating):
    '''create a dictionary with precise rating score'''
    precise_rating = {}
    for museum_name, rating_lst in traveler_rating.items():
        # if the rating list is [13, 10, 3, 4, 2]
        # then the rating sum is 13*5 + 10*4 + 3*3 + 4*2 + 2*1
        rating_sum = reduce(lambda x,y: x+y , [int(val)*(5-idx) for idx, val in enumerate(rating_lst)])
        # get review count of the museum
        review_count = merged_df.loc[df[df['MuseumName'] == museum_name].index.tolist()[0], 'ReviewCount']
        # calculate precise rating
        precise_rating[museum_name] = rating_sum/review_count
    return precise_rating

### Read Data 

In [31]:
# read musuem data (.csv)
# usa
us_museum = pd.read_csv("./Data/tripadvisor_museum_USonly.csv")
us_category = read_json('./Data/museum_categories_USonly.json')
us_review = read_json('./Data/review_content_USonly.json')
us_quote = read_json('./Data/review_quote_USonly.json')
us_tag_cloud = read_json('./Data/tag_clouds_USonly.json')
us_traveler_type = read_json('./Data/traverler_type_USonly.json')
us_traveler_rating = read_json('./Data/traverler_rating_USonly.json')

# world
w_museum = pd.read_csv("./Data/tripadvisor_museum_world.csv")
w_category = read_json('./Data/museum_categories_world.json')
w_review = read_json('./Data/review_content_world.json')
w_quote = read_json('./Data/review_quote_world.json')
w_tag_cloud = read_json('./Data/tag_clouds_world.json')
w_traveler_type = read_json('./Data/traverler_type_world.json')
w_traveler_rating = read_json('./Data/traverler_rating_world.json')

# read USA state full/abbr name
state_name = pd.read_csv("./Data/states.csv")
state_name_lst = state_name['Abbreviation'].tolist()

In [32]:
us_museum.columns.values 
# an unknown first column is in the dataframe
# thus we drop the first column later on

array(['Unnamed: 0', 'Address', 'Description', 'FeatureCount', 'Fee',
       'Langtitude', 'Latitude', 'LengthOfVisit', 'MuseumName', 'PhoneNum',
       'Rank', 'Rating', 'ReviewCount', 'TotalThingsToDo'], dtype=object)

In [33]:
# discover that....the value in lengthOfVisit has value should be in column 'Fee'....
set(us_museum['LengthOfVisit'])

{nan,
 '1-2 hours ',
 '2-3 hours ',
 '<1 hour ',
 'More than 3 hours ',
 'No ',
 'Yes '}

### Data Merging, Cleaning and Adding New Features

In [52]:
# drop the first column
us_museum, w_museum = drop_first_column(us_museum, w_museum)

# merge usa and world museum
merged_df = merge_usa_world(us_museum, w_museum)
nrows = merged_df.shape[0]

# fix the lengthOfVsiit and fee value
# merged_df = fix_fee_value(merged_df)
# merged_df = merged_df.drop(merged_df.index[range(nrows, merged_df.shape[0])]) # drop exrta rows created
# not sure why the function "fix_fee_value" wont work 
idx_no = merged_df[merged_df['LengthOfVisit'] == 'No '].index.tolist()
idx_yes = merged_df[merged_df['LengthOfVisit'] == 'Yes '].index.tolist()
df = merged_df.copy(deep = True)
for idx in idx_no:
    df.loc[idx,'LengthOfVisit'] = None
for idx in idx_yes:
    df.loc[idx,'LengthOfVisit'] = None
merged_df = df.copy(deep = True)
print set(merged_df['LengthOfVisit'])
print merged_df.shape

# convert string to number
merged_df = string_to_num(merged_df, ['ReviewCount','TotalThingsToDo'])

# add new feature "Country": USA or Other & "State": state name or Not_in_USA
add_country_state(merged_df, state_name_lst)
merged_df = merged_df.drop(merged_df.index[range(nrows, merged_df.shape[0])]) # drop exrta rows created

# add feature 'RankPercentage'
merged_df['RankPercentage'] = merged_df['Rank']*100/merged_df['TotalThingsToDo'] 
merged_df.shape # check the shape

set([nan, '2-3 hours ', '<1 hour ', None, 'More than 3 hours ', '1-2 hours '])
(1487, 13)


(1487, 16)

In [53]:
# combind USA and world dictionaries
category = append_two_dict(us_category, w_category)
review = append_two_dict(us_review, w_review)
quote = append_two_dict(us_quote, w_quote)
tag_cloud = append_two_dict(us_tag_cloud, w_tag_cloud)
traveler_type = append_two_dict(us_traveler_type, w_traveler_type)
traveler_rating = append_two_dict(us_traveler_rating, w_traveler_rating)

# convert all dictionaries from unicode to ascii
category = encode_whole_dictionary(category)
review = encode_whole_dictionary(review)
quote = encode_whole_dictionary(quote)
tag_cloud = encode_whole_dictionary(tag_cloud)
traveler_type = encode_whole_dictionary(traveler_type)
traveler_rating = encode_whole_dictionary(traveler_rating)

# convert strings in dictionary to number
traveler_type = dic_str_to_num(traveler_type)
traveler_rating = dic_str_to_num(traveler_rating)

In [54]:
# add text-based features by assigning 1 or 0 
# (if a museum matches or doesn't match the criteria)
merged_df = add_category_feature(merged_df, category)
merged_df = add_tags_feature(merged_df, tag_cloud)
merged_df = add_traveler_type_count(merged_df)
# merged_df = string_to_num(merged_df, ['Families_Count','Couples_Count', 'Solo_Count', 'Business_Count', 'Friends_Count'])

In [55]:
# calculate precise rating and add it as new column
# yet to decide whether I should predict the displayed rating for precise rating...
precise_rating_dic = get_precise_rating(merged_df, traveler_rating)
merged_df = add_one_feature(merged_df, 'PreciseRating', precise_rating_dic)

### Add More New Features - Sentiment Analysis Scores

In [56]:
# create description dictionary {museum name: description}
description_dic = create_description_dic(merged_df)

In [57]:
# get sentiment scores for quote/review/description
quote_polarity_dic, quote_subjectivity_dic = get_nested_sentiment(quote)
review_polarity_dic, review_subjectivity_dic = get_nested_sentiment(review)
des_polarity_dic, des_subjectivity_dic = get_text_sentiment(description_dic)

In [58]:
merged_df = add_multiple_score_feature(merged_df, 'quote_pol_', quote_polarity_dic)
merged_df = add_multiple_score_feature(merged_df, 'quote_sub_', quote_subjectivity_dic)
merged_df = add_multiple_score_feature(merged_df, 'review_pol_', review_polarity_dic)
merged_df = add_multiple_score_feature(merged_df, 'review_sub_', review_subjectivity_dic)

In [59]:
# somehow the dictionary has a float('NaN') as key... remove the key from dictionary
des_polarity_dic = clean_dic(des_polarity_dic)
des_subjectivity_dic = clean_dic(des_subjectivity_dic)

# add sentiment score of museum description as new features
merged_df = add_one_feature(merged_df, 'descri_pol', des_polarity_dic)
merged_df = add_one_feature(merged_df, 'descri_sub', des_subjectivity_dic)
merged_df = merged_df.drop(merged_df.index[range(nrows, merged_df.shape[0])]) # drop exrta rows created

In [60]:
merged_df.shape

(1487, 82)

In [61]:
merged_df.to_csv('tripadvisor_merged.csv')

In [62]:
merged_df.columns.values

array(['Address', 'Description', 'FeatureCount', 'Fee', 'Langtitude',
       'Latitude', 'LengthOfVisit', 'MuseumName', 'PhoneNum', 'Rank',
       'Rating', 'ReviewCount', 'TotalThingsToDo', 'Country', 'State',
       'RankPercentage', 'History Museums', 'Military Museums',
       'Points of Interest & Landmarks', 'Natural History Museums',
       'Art Museums', 'Historic Sites', 'Science Museums', 'Museums',
       'Specialty Museums', 'Art Galleries', "Children's Museums",
       'Sights & Landmarks', 'on display', 'gift shop', 'rainy day',
       'all ages', 'couple of hours', 'special exhibits', 'few hours',
       'permanent collection', 'two hours', 'exhibits', 'worth a visit',
       'free admission', 'beautiful building', 'audio guide',
       'well worth a visit', 'great collection', 'information',
       'interactive exhibits', 'great for kids', 'hands on activities',
       'interesting exhibits', 'great exhibits', 'entrance fee',
       'great place to visit', 'well worth t

### Miscellaneous Note/Code

In [63]:
'Feature Name'.replace(' ', '_')

'Feature_Name'

In [64]:
# data exploratory use
tag_freq_lst = get_tag_cloud_lst(tag_cloud)
target_tags = tag_freq_lst[0:30]
target_tags

[('on display', 751),
 ('gift shop', 405),
 ('rainy day', 281),
 ('all ages', 275),
 ('couple of hours', 268),
 ('special exhibits', 188),
 ('few hours', 167),
 ('permanent collection', 162),
 ('two hours', 158),
 ('exhibits', 155),
 ('worth a visit', 154),
 ('free admission', 151),
 ('beautiful building', 136),
 ('audio guide', 135),
 ('well worth a visit', 130),
 ('great collection', 125),
 ('information', 117),
 ('interactive exhibits', 112),
 ('great for kids', 111),
 ('hands on activities', 111),
 ('interesting exhibits', 109),
 ('great exhibits', 108),
 ('entrance fee', 104),
 ('great place to visit', 100),
 ('well worth the visit', 85),
 ('guided tour', 81),
 ('amazing collection', 81),
 ('traveling exhibits', 79),
 ('interactive displays', 79),
 ('whole family', 78)]

In [65]:
# However...I am not sure why precise ratings are so different from
# the ratings displayed on Tripadvisor (maybe the metric is different?)
test_df = pd.DataFrame({'precise_rating':merged_df['PreciseRating'], 
                        'rating': merged_df['Rating'], 
                        'museum': merged_df['MuseumName']})
test_df.head(10)

,museum,precise_rating,rating
0,Newseum,4.471638,4.5
1,The Metropolitan Museum of Art,2.747520,5.0
2,The National WWII Museum,4.601754,5.0
3,Denver Museum of Nature & Science,4.450138,4.5
4,Art Institute of Chicago,4.143376,5.0
5,The National 9/11 Memorial & Museum,3.137601,4.5
6,Museum of Fine Arts,0.463276,4.5
7,The Field Museum,3.869613,4.5
8,Smithsonian National Museum of Natural History,3.784190,4.5
9,Smithsonian National Air and Space Museum,3.470940,4.5
